In [9]:
import numpy as np
import open3d.ml.torch as ml3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Exception: Version mismatch: Open3D needs PyTorch version 1.8.*, but version 1.10.0+cu113 is installed!

In [ ]:
data = [ {
    'name': 'my_point_cloud',
    'points': np.random.rand(100,3).astype(np.float32),
    'point_attr1': np.random.rand(100).astype(np.float32),
    } ]

vis = ml3d.vis.Visualizer()
vis.visualize(data)

In [ ]:
import open3d as o3d
import open3d.visualization.gui as gui

app = gui.Application.instance
app.initialize()

w = app.create_window("Open3D - 3D Labels", 1024, 768)
widget3d = gui.SceneWidget()
w.add_child(widget3d)
widget3d.scene = o3d.visualization.rendering.Open3DScene(w.renderer)

# Add torus
torus = o3d.geometry.TriangleMesh.create_torus()
torus.compute_vertex_normals()
#mat = o3d.visualization.rendering.Material()
#mat.shader = "defaultLit"
widget3d.scene.add_geometry("Torus", torus)#, mat)

# Add torus vertices
pts = o3d.geometry.PointCloud(torus.vertices)
#mat = o3d.visualization.rendering.Material()
#mat.shader = "defaultUnlit"
#mat.point_size = 5 * w.scaling
#mat.base_color = (0, 0, 0, 1)
widget3d.scene.add_geometry("Points", pts)#, mat)


# Add 3D labels
for i in range(0, len(torus.vertices)):
    widget3d.add_3d_label(torus.vertices[i], str(i))

bounds = widget3d.scene.bounding_box
widget3d.setup_camera(60, bounds, bounds.get_center())

app.run()

In [9]:
def make_point_cloud(npts, center, radius):
    pts = np.random.uniform(-radius, radius, size=[npts, 3]) + center
    cloud = o3d.geometry.PointCloud()
    cloud.points = o3d.utility.Vector3dVector(pts)
    colors = np.random.uniform(0.0, 1.0, size=[npts, 3])
    cloud.colors = o3d.utility.Vector3dVector(colors)
    return cloud


def high_level():
    app = gui.Application.instance
    app.initialize()
    print("Initializied Open3D GUI")
    points = make_point_cloud(1, (0, 0, 0), 1.0)
    print("Initializied Open3D Point Cloud")
    vis = o3d.visualization.O3DVisualizer("Open3D - 3D Text", 1024, 768)
    vis.show_settings = True
    vis.add_geometry("Points", points)
    for idx in range(0, len(points.points)):
        vis.add_3d_label(points.points[idx], "{}".format(idx))

    vis.reset_camera_to_default()
    app.add_window(vis)
    app.run()
high_level()

Initializied Open3D GUI
Initializied Open3D Point Cloud
[Open3D INFO] Window window_6 created.


KeyboardInterrupt: 

In [108]:
from itertools import compress
import torch
pred_dict = {"pred_boxes": torch.rand((20,9)), "pred_labels": torch.randint(0,10,(20,1)),"pred_scores": torch.rand((20,1))}
#%timeit list(compress(pred_dict,pred_dict["pred_labels"]==9))
#%timeitnp.array(list(compress(pred_dict,pred_dict["pred_labels"]==9)))

In [13]:
%timeit np.array(list(compress(pred_dict,pred_dict["pred_labels"]==9)))


1.68 µs ± 10.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [24]:
%timeit (np.array(list(compress(pred_dict["pred_boxes"],pred_dict["pred_labels"] in [9]))))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [110]:
classes = [5,4,6,4]
#%timeit indices = [np.nonzero(sum(pred_dict["pred_labels"]==x for x in classes))[0].tolist()]

In [111]:
indices = [np.nonzero(sum(pred_dict["pred_labels"]==x for x in classes))[0].tolist()]
%timeit pred_dict["pred_boxes"][indices,:].tolist()

7.27 µs ± 112 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [83]:
import json
import time
from copy import copy

In [175]:
def send():
    pred_dict = {"pred_boxes": torch.rand((20,9)), "pred_labels": torch.randint(0,10,(20,1)),"pred_scores": torch.rand((20,1))}
    if isinstance(pred_dict["pred_labels"],torch.Tensor):
        pred_dict["pred_labels"] = pred_dict["pred_labels"].cpu().numpy()
    classes = [4,5,6]
    indices = np.nonzero(sum(pred_dict["pred_labels"]==x for x in classes))[0].tolist()
    pred_dict["pred_boxes"] = pred_dict["pred_boxes"][indices,:].tolist()
    pred_dict["pred_labels"] = pred_dict["pred_labels"][indices,:].tolist()
    pred_dict["pred_scores"] = pred_dict["pred_scores"][indices,:].tolist()
    for key,value in pred_dict.items():
        print(np.shape(value))
    data = json.dumps(pred_dict).encode('utf-8')
send()

(6, 9)
(6, 1)
(6, 1)


In [170]:
pred_dict = {"pred_boxes": torch.rand((20,9)), "pred_labels": np.random.randint(0,10,(20,1)),"pred_scores": torch.rand((20,1))}
if isinstance(pred_dict["pred_labels"],torch.Tensor):
    pred_dict["pred_labels"] = pred_dict["pred_labels"].cpu().numpy()
pcd = np.random.random((80000,3))
classes = [1,2,3,4,5,6,7,8]
start = time.time()
indices = np.nonzero(sum(pred_dict["pred_labels"]==x for x in classes))[0].tolist()
print(indices)
#print(indices)
#print(pred_dict["pred_boxes"][indices,:].shape)
#print(pred_dict["pred_labels"][indices,:].shape)
#print(pred_dict["pred_scores"][indices,:].shape)

pred_send = np.concatenate((pred_dict["pred_boxes"][indices,:],pred_dict["pred_labels"][indices,:],pred_dict["pred_scores"][indices,:]),axis=1)
print(pred_send.shape)
print(time.time()-start)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
(19, 11)
0.0009610652923583984


In [155]:
pcd = np.random.random((80000,3))

55.4 µs ± 530 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


1805.0541516245487